In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2

k = 2 * np.pi / 0.5
Omega = 0.5
epsilon = 1e-10

def E_d(A, phi, x, y, x_prime, y_prime):
    r = np.sqrt((x - x_prime)**2 + (y - y_prime)**2) + epsilon 
    return A / r * np.exp(-1j * (k * r) + 1j * phi)

def E_i(E_d_values):
    return E_d_values * (1 + np.cos(Omega))

size = 64
wavelength = 2.14e-3
distance = 50e-3

x = np.linspace(-5, 5, size)
y = np.linspace(-5, 5, size)
X, Y = np.meshgrid(x, y)

# input and target distribution
center_x = size // 2
center_y = size // 2
sigma = size / 5
x, y = np.meshgrid(np.arange(size), np.arange(size))
input_intensity = np.exp( - ((x - center_x)**2 + (y - center_y)**2) / (2 * sigma**2) ) 
input_intensity /= np.sum(input_intensity)

target_intensity = np.zeros((size, size))
width = size // 48
center_x = size // 2
center_y = size // 2
target_intensity[center_x - width: center_x, 24: 40] = 1.0
target_intensity[24: 40, center_y - width: center_y] = 1.0

def propagate(field, distance, wavelength, size, X, Y):
    k = 2 * np.pi / wavelength
    propagated_field = np.zeros((size, size), dtype=np.complex128)
    
    for x_prime in range(size):
        for y_prime in range(size):
            r = np.sqrt((X - X[x_prime, y_prime])**2 + (Y - Y[x_prime, y_prime])**2 + distance**2) + epsilon
            H = (1 / r) * np.exp(-1j * k * r) * (1 + np.cos(np.arctan(distance / r)))
            propagated_field[x_prime, y_prime] = np.sum(field * H)
    
    return propagated_field

def gerchberg_saxton(input_intensity, target_intensity, wavelength, distance, iterations=100):
    size = input_intensity.shape[0]
    X, Y = np.meshgrid(np.linspace(-5, 5, size), np.linspace(-5, 5, size))
    
    # Initialize the phase and field distributions
    phase = np.random.rand(size, size)
    input_field = np.sqrt(input_intensity) * np.exp(1j * phase)

    for i in range(iterations):
        # Forward propagation
        E_d_values = np.zeros_like(input_field, dtype=complex)
        for x_prime in range(size):
            for y_prime in range(size):
                E_d_values[x_prime, y_prime] = E_d(np.sqrt(input_intensity[x_prime, y_prime]), phase[x_prime, y_prime], X[x_prime, y_prime], Y[x_prime, y_prime], X[x_prime, y_prime], Y[x_prime, y_prime])
        E_i_values = E_i(E_d_values)
        output_field = propagate(E_i_values, distance, wavelength, size, X, Y)
        
        # Enforce the target intensity pattern
        output_phase = np.angle(output_field)
        output_field = np.sqrt(target_intensity) * np.exp(1j * output_phase)
        
        # Backward propagation
        E_d_values = np.zeros_like(output_field, dtype=complex)
        for x_prime in range(size):
            for y_prime in range(size):
                E_d_values[x_prime, y_prime] = E_d(np.sqrt(target_intensity[x_prime, y_prime]), output_phase[x_prime, y_prime], X[x_prime, y_prime], Y[x_prime, y_prime], X[x_prime, y_prime], Y[x_prime, y_prime])
        E_i_values = E_i(E_d_values)
        input_field = propagate(E_i_values, -distance, wavelength, size, X, Y)
        
        # Enforce the input intensity pattern
        input_phase = np.angle(input_field)
        input_field = np.sqrt(input_intensity) * np.exp(1j * input_phase)

    # Calculate the phase plate
    phase_plate = np.angle(input_field)
    
    return phase_plate

phase_plate = gerchberg_saxton(input_intensity, target_intensity, wavelength, distance)

plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.title('Input Intensity')
plt.imshow(input_intensity, cmap='gray')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.title('Target Intensity')
plt.imshow(target_intensity, cmap='gray')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.title('Phase Plate')
plt.imshow(phase_plate, cmap='gray')
plt.colorbar()

plt.show()

print(phase_plate)

def project_with_phase_plate(source_field, phase_plate, distance, wavelength, size, X, Y):
    """
    Simulates the effect of a phase plate on a source beam and projects the result.

    Args:
      source_field: Complex NumPy array representing the source beam wavefront.
      phase_plate: NumPy array representing the phase information of the phase plate.
      distance: Propagation distance between the phase plate and the image plane (meters).
      wavelength: Wavelength of the light used (meters).
      size: Size of the arrays (assumes square shape).
      X, Y: Meshgrids representing spatial coordinates within the arrays.

    Returns:
      Complex NumPy array representing the propagated wavefront at the image plane.
    """
    X, Y = np.meshgrid(np.linspace(-5, 5, size), np.linspace(-5, 5, size))
#     X, Y = np.meshgrid(x, y)

    # Apply phase plate
    

    # Propagate the modified field
    propagated_input_field = propagate(source_field, 30e-3, wavelength, size, X, Y)

    modified_field = source_field * np.exp(1j * phase_plate)
    
    propagated_image = propagate(modified_field, 40e-3, wavelength, size, X, Y)
    
    intensity_image = np.abs(propagated_image)**2
    # Return the propagated field (represents the image)
    return intensity_image

intensity_image = project_with_phase_plate(input_intensity, phase_plate, distance, wavelength, size, X, Y)

plt.figure(figsize=(8, 6))
plt.title('Simulated Intensity')
plt.imshow(intensity_image, cmap='gray')
plt.colorbar()
plt.show()